In [1]:
# Adding the parent directory to path so that 
# skexplain can be imported without being explicitly
import os,sys
path = os.path.dirname(os.getcwd())
sys.path.append(path)

_base_module_path = '/home/monte.flora/python_packages/master/WoF_post'
sys.path.append(_base_module_path)

from wofs_ml_severe.data_pipeline.storm_based_feature_extracter import StormBasedFeatureExtracter
from wofs.post.utils import load_yaml
import numpy as np
from scipy.stats import multivariate_normal
import monte_python
import xarray as xr 
import glob
from numba import jit

In [2]:
ml_config_dict = load_yaml(os.path.join(path,'wofs_ml_severe/conf','test_config.yml'))

In [3]:
extracter = StormBasedFeatureExtracter(ml_config_dict)

In [4]:
ds = xr.load_dataset('test_ENSEMBLETRACKS.nc')

In [5]:
# Extract the spatial statistics. 
storm_objects = ds.label_img.values
intensity_img = ds.intensity_img.values

object_props_df = extracter.get_object_properties(storm_objects, intensity_img)
labels = object_props_df['label'].values


In [22]:
# Extract the amplitude statistics (conditional statistics). 

# Build a fake dataset (NE, NY, NX)
NE = 5
data = np.zeros((NE, storm_objects.shape[0], storm_objects.shape[1]))
fake_data = [1,3,5,7,9]
for i in range(NE):
    for d, label in zip(fake_data, labels):
        data[i, storm_objects==label] = d+i
    
# Build a conditional variable 
cond_data = np.zeros((NE, storm_objects.shape[0], storm_objects.shape[1]))
for i in range(NE):
    if i > 2:
        for label in labels:
            cond_data[i, storm_objects==label] = 2
    
storm_data_time_composite = {'ens_var1__time_max' : data, 
                             'cond_var' : cond_data,
                            }
    
df_amp = extracter.extract_amplitude_features_from_object( 
                storm_data_time_composite, storm_objects, labels, cond_var='cond_var', 
                cond_var_thresh = 1)

vals = np.unique(data[:,storm_objects==1])
true_arr = [np.std(vals, ddof=1), np.mean(vals), np.max(vals)]

# This should be true! 
# This ensures that though we are computing the conditional variables
# we are still correctly computing the marginal variables. 
cols = ['ens_var1__time_max__amp_ens_std', 'ens_var1__time_max__amp_ens_mean', 'ens_var1__time_max__amp_ens_max' ]
np.round(df_amp.loc[0,cols].values,2) - np.round(true_arr,2) < 1e-7

# Computed the conditional statistics correctly? 
vals = np.unique(data[:,storm_objects==1])[3:]
stats = [np.std(vals, ddof=1), np.mean(vals)]

cols = [f'{c}__cond' for c in cols if 'ens_max' not in c ]
np.round(df_amp.loc[0,cols].values,2) - np.round(stats,2) < 1e-7

n_features=10 n_vars=2 n_stats=3


array([ True,  True])